# DATA READING

In [0]:
df_transform = spark.read.format("parquet").option("InferSchema", True).load("abfss://bronze@carprojectdatalakejess.dfs.core.windows.net/raw_data")

In [0]:
from pyspark.sql.functions import *


ADDING NEW COLUMN BASED ON THE DIVISON OF REVENUE AND UNITS_SOLD DATA

In [0]:
df_transform = df_transform.withColumn("RevPerUnit", col("Revenue") / col("Units_Sold"))
df_transform.display()


SPLITTING DATA IN THE MODEL ID COLUMN

In [0]:

df_transform = df_transform.withColumn("Model_Category", split(col("Model_ID"), "-")[0])\
.withColumn("Model_ID", split(col("Model_ID"), "-")[1])
df_transform.display()

AD HOC(Testing Out Data For Analysis)

In [0]:
df_transform.groupBy("Year", "BranchName").agg(sum("Units_Sold").alias("Total_Units_Sold")).sort("Year", "Total_Units_Sold").display()

Databricks visualization. Run in Databricks to view.

# DATA WRITING

In [0]:
df_transform.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "abfss://silver@carprojectdatalakejess.dfs.core.windows.net/car_sales") \
    .save()



# Querying Silver Data

In [0]:
%sql
SELECT * FROM parquet.`abfss://silver@carprojectdatalakejess.dfs.core.windows.net/car_sales`